In [1]:
import librosa
import librosa.display
import scipy as sp
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import statistics as s
import math

song_path = 'pps.mp3' # Iron Lotus boiiiiiiiiiiiiiiiiii~~
song, sr = librosa.load(song_path)

C:\Users\scarl\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [2]:
import colorsys

NOTES = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

def note2hue(note, noteList, rtrn="hexa"):
    index = -1
    if('#' in note):
        index = noteList.index(note[:2])
    else:
        index = noteList.index(note[0])
    hue = (index/12)
    rgb = colorsys.hsv_to_rgb(hue,0.9,0.8)
    rgb = (rgb[0]*255,rgb[1]*255,rgb[2]*255)
    rgb = (math.floor(rgb[0]),math.floor(rgb[1]),math.floor(rgb[2]))
    if(rtrn == 'rgb'):
        return r
    hexa = '#%02x%02x%02x' % rgb #how to get hexadecimal from rgb
    return hexa

def allNote2Hue(allMidi, noteList):
    allNote = []
    for m in allMidi:
        temp = m
        try:
            temp['colour'] = note2hue(m['midi'],noteList,rtrn='hexa')
        except:
            temp['colour'] = '-'
        allNote.append(temp)
    return allNote

In [3]:
def midi2hue(midi, amp, maxAmp, rtrn="hexa"):
    hue = (midi%12)/12
    val = 0.70+(midi/96)*0.30
    sat = 0.70-(midi/96)*0.30
    # Need to incorporate Amp --------------------
#     print(amp,maxAmp)
    ampp = 0.20 * amp/maxAmp
    sat -= ampp
    val += ampp
    val = 1 if val>1 else val

    rgb = colorsys.hsv_to_rgb(hue,sat,val)
    rgb = (rgb[0]*255,rgb[1]*255,rgb[2]*255)
    rgb = (math.floor(rgb[0]),math.floor(rgb[1]),math.floor(rgb[2]))
    if(rtrn == 'rgb'):
        return rgb
    hexa = '#%02x%02x%02x' % rgb #how to get hexadecimal from rgb
    return hexa

def allMidi2Hue(allMidi):
    allNote = []
    maxAmp = max(allMidi, key=lambda x:x['amp'])['amp']
    for m in allMidi:
        temp = m
        try:
            temp['colour'] = midi2hue(m['midi'],m['amp'],maxAmp)
        except:
            temp['colour'] = '-'
        allNote.append(temp)
    return allNote

In [4]:
def getTempo(song,sr):
    onset_env = librosa.onset.onset_strength(y=song, sr=sr)
    tempo = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    return tempo #in bpm

def secondsPerBeat(tempo):
    secPerBeat = 60 / tempo #60 seconds per beat
    return secPerBeat

def getFreq(signal, sr):
    ft = sp.fft.fft(signal)
    magnitude = np.absolute(ft)
    freq = np.linspace(0,sr,len(magnitude))
    return freq, magnitude

def getRms(song,spb):
    FRAME_LEN = spb*2
    HOP_LEN = spb

    rms = librosa.feature.rms(song,frame_length=FRAME_LEN,hop_length=HOP_LEN)[0]
    
    return rms

In [5]:
def getMidi(magnitude):
    index = np.argmax(magnitude)
    try:
        midi = librosa.hz_to_midi(index)
    except:
        midi = '-'
    return midi

def getMidiAll(signal,sr,spb):
    # all calculated in sample length
    songDuration = len(signal)
    samplePerBeat = math.ceil(spb*sr)
    rms = getRms(signal,samplePerBeat)
    print(max(rms))
    print(s.mean(rms))
    allMidi = []
    for p in range(0,songDuration,samplePerBeat):
        oneBeat = []
        if(p+samplePerBeat < songDuration):
            oneBeat = signal[p:p+samplePerBeat]
        else:
            oneBeat = signal[p:]
        freq,magnitude = getFreq(oneBeat,sr)
        midi = getMidi(magnitude)
        allMidi.append({'start':round((p)*(1/sr),2),'end':round((p+samplePerBeat)*(1/sr),2),'midi':midi})
    for i in range(len(allMidi)):
        allMidi[i]['amp'] = rms[i]
    return allMidi # seconds, seconds, note

In [6]:
def getAllMidi(song,sr):
    tempo = getTempo(song,sr)
    spb = secondsPerBeat(tempo)
    allMidi = getMidiAll(song,sr,spb)
    allMidi = allMidi2Hue(allMidi)
    return allMidi

In [7]:
allMidi = getAllMidi(song,sr)
for i in allMidi:
    print(i)

<ipython-input-4-c7523b52a756>:20: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  rms = librosa.feature.rms(song,frame_length=FRAME_LEN,hop_length=HOP_LEN)[0]


0.4133961
0.2775134
{'start': 0.0, 'end': 0.39, 'midi': 40.331492511049206, 'amp': 0.12913461, 'colour': '#6ee281'}
{'start': 0.39, 'end': 0.79, 'midi': 39.048510064051065, 'amp': 0.14301327, 'colour': '#a6e36f'}
{'start': 0.79, 'end': 1.18, 'midi': 39.26905241582932, 'amp': 0.08103421, 'colour': '#90db65'}
{'start': 1.18, 'end': 1.58, 'midi': 39.26905241582932, 'amp': 0.075897716, 'colour': '#8fdb64'}
{'start': 1.58, 'end': 1.97, 'midi': 39.26905241582932, 'amp': 0.082274646, 'colour': '#90db65'}
{'start': 1.97, 'end': 2.37, 'midi': 39.26905241582932, 'amp': 0.084000275, 'colour': '#91dc66'}
{'start': 2.37, 'end': 2.76, 'midi': 39.26905241582932, 'amp': 0.08255086, 'colour': '#90db65'}
{'start': 2.76, 'end': 3.16, 'midi': 39.26905241582932, 'amp': 0.0831638, 'colour': '#91dc65'}
{'start': 3.16, 'end': 3.55, 'midi': 40.331492511049206, 'amp': 0.114387415, 'colour': '#6ce07f'}
{'start': 3.55, 'end': 3.95, 'midi': 40.331492511049206, 'amp': 0.13843958, 'colour': '#70e383'}
{'start': 3.95

In [8]:
# ipd.Audio(song_path, Autoplay = True)
rms = getRms(song)
print(len(rms))
print(len(song))

TypeError: getRms() missing 1 required positional argument: 'spb'